# 10 - Engenharia de Features Avançada

Este notebook cria features avançadas baseadas nos dados de treino e validação temporal.

## Novas Features:
- **Target de Classificação**: `vendeu` (1 se quantidade > 0, 0 caso contrário)
- **Features de Preço**: preço unitário, variações, médias
- **Features de Calendário**: dia do mês, semana do mês, flags temporais
- **Features de Hierarquia**: agregações por categoria, zipcode
- **Features de Tendência**: momentum, aceleração de vendas

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import warnings
warnings.filterwarnings('ignore')

print('🚀 Iniciando Engenharia de Features Avançada')
print('🎯 Objetivo: Criar variáveis preditivas poderosas para o modelo de dois estágios')

# Criar pasta submissao3 se não existir
os.makedirs('../data/submissao3', exist_ok=True)
print('📁 Pasta data/submissao3 criada/verificada')

🚀 Iniciando Engenharia de Features Avançada
🎯 Objetivo: Criar variáveis preditivas poderosas para o modelo de dois estágios
📁 Pasta data/submissao3 criada/verificada


## 1. Carregamento dos Dados de Treino e Validação

In [2]:
# Carregar dados de treino e validação
print('📂 Carregando dados de treino e validação...')

train_data = pd.read_parquet('../data/submissao3/train_data.parquet')
validation_data = pd.read_parquet('../data/submissao3/validation_data.parquet')

print(f'🏋️ Dados de treino: {train_data.shape}')
print(f'🔍 Dados de validação: {validation_data.shape}')

# Verificar período
print(f'📅 Treino: {train_data["semana"].min()} até {train_data["semana"].max()}')
print(f'📅 Validação: {validation_data["semana"].min()} até {validation_data["semana"].max()}')

print('✅ Dados carregados com sucesso')

📂 Carregando dados de treino e validação...
🏋️ Dados de treino: (50126880, 6)
🔍 Dados de validação: (5221550, 6)
📅 Treino: 2021-12-28 00:00:00 até 2022-11-22 00:00:00
📅 Validação: 2022-11-29 00:00:00 até 2022-12-27 00:00:00
✅ Dados carregados com sucesso


## 2. Carregamento dos Dados Auxiliares

In [3]:
# Carregar dados de produtos e PDVs para features adicionais
print('📂 Carregando dados auxiliares...')

# Produtos
produtos = pd.read_parquet(
    '../data/part-00000-tid-7173294866425216458-eae53fbf-d19e-4130-ba74-78f96b9675f1-4-1-c000.snappy.parquet'
)
produtos = produtos.rename(columns={
    'produto': 'produto_id'
})

# PDVs
pdvs = pd.read_parquet(
    '../data/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet'
)
pdvs = pdvs.rename(columns={
    'pdv': 'pdv_id',
    'categoria_pdv': 'tipo_loja'
})

print(f'📊 Produtos: {produtos.shape} - Categorias: {produtos["categoria"].nunique()}')
print(f'📊 PDVs: {pdvs.shape} - Zipcodes: {pdvs["zipcode"].nunique()}')

print('✅ Dados auxiliares carregados')

📂 Carregando dados auxiliares...
📊 Produtos: (7092, 8) - Categorias: 7
📊 PDVs: (14419, 4) - Zipcodes: 788
✅ Dados auxiliares carregados


## 3. Função de Engenharia de Features

In [4]:
# =================================================================================
# FUNÇÃO BLINDADA CONTRA CRASH DE MEMÓRIA - DOWNCASTING INCREMENTAL SEGURO
# =================================================================================

def reduce_mem_usage(df, name):
    """
    Reduz o uso de memória de um dataframe, alterando os tipos de dados das colunas.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'🔄 Otimizando memória para {name}...')
    print(f'   📊 Memória inicial: {start_mem:.2f} MB')

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object and col_type.name != 'category' and 'datetime' not in col_type.name:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                # Para floats, vamos usar float32 que é suficiente para a maioria dos casos
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'   ✅ Memória final: {end_mem:.2f} MB ({100 * (start_mem - end_mem) / start_mem:.1f}% de redução)')
    return df

def safe_downcast_column(series, target_dtype):
    """
    Faz downcast seguro de uma coluna, verificando limites antes.
    Converte strings para numérico se necessário.
    """
    # Se for string, converter para numérico primeiro
    if series.dtype == 'object':
        try:
            series = pd.to_numeric(series, errors='coerce')
            print(f'   🔄 Coluna {series.name}: convertida de string para numérico')
        except:
            print(f'   ⚠️ Coluna {series.name}: não foi possível converter para numérico')
            return series
    
    if target_dtype == 'int32':
        if series.min() >= np.iinfo(np.int32).min and series.max() <= np.iinfo(np.int32).max:
            return series.astype('int32')
        else:
            print(f'   ⚠️ Coluna {series.name}: valores muito grandes para int32, mantendo int64')
            return series.astype('int64')
    elif target_dtype == 'float32':
        if series.min() >= np.finfo(np.float32).min and series.max() <= np.finfo(np.float32).max:
            return series.astype('float32')
        else:
            return series.astype('float64')
    else:
        return series.astype(target_dtype)

def criar_features_avancadas_otimizada(dados, dados_produtos, dados_pdvs, nome_conjunto="DATASET"):
    """
    Cria features avançadas com downcasting incremental SEGURO para controle máximo de memória.
    VERSÃO BLINDADA: Otimiza memória a cada etapa para evitar crash, verificando limites.
    VERSÃO CORRIGIDA: Remove target leakage das features rolantes.
    """
    print(f'\\n🔧 Criando features avançadas para {nome_conjunto} (VERSÃO CORRIGIDA - SEM TARGET LEAKAGE)...')
    
    # ============================================================================
    # ETAPA 1: OTIMIZAÇÃO INICIAL DOS DADOS BASE (SEGURA)
    # ============================================================================
    print('   🎯 Otimização inicial de tipos de dados (verificando limites)...')
    
    # Verificar e downcast de IDs com segurança
    for col in ['pdv_id', 'produto_id', 'distributor_id']:
        if col in dados.columns:
            print(f'      🔍 {col}: tipo={dados[col].dtype}')
            # Se for string, mostrar alguns valores de exemplo
            if dados[col].dtype == 'object':
                print(f'         Exemplos: {dados[col].head(3).tolist()}')
            else:
                print(f'         Range: {dados[col].min()} até {dados[col].max()}')
            dados[col] = safe_downcast_column(dados[col], 'int32')
    
    if 'quantidade' in dados.columns:
        dados['quantidade'] = safe_downcast_column(dados['quantidade'], 'int32')
    if 'faturamento' in dados.columns:
        dados['faturamento'] = safe_downcast_column(dados['faturamento'], 'float32')
    
    print('   ✅ Otimização inicial concluída.')
    
    # ============================================================================
    # ETAPA 2: TARGET E MERGES COM DOWNCASTING IMEDIATO
    # ============================================================================
    print('   🎯 Criando target e fazendo merge com dados auxiliares...')
    
    # Target como int8 (muito mais eficiente)
    dados['vendeu'] = (dados['quantidade'] > 0).astype('int8')
    
    # Merge com dados auxiliares
    dados = dados.merge(dados_produtos[['produto_id', 'categoria']], on='produto_id', how='left')
    dados = dados.merge(dados_pdvs[['pdv_id', 'zipcode', 'tipo_loja']], on='pdv_id', how='left')
    
    # CORREÇÃO: Verificar se semana_original existe antes de sobrescrever
    if 'semana_original' not in dados.columns:
        dados['semana_original'] = dados['semana']
        print('   📅 Coluna semana_original criada para preservar separação treino/validação')
    
    # Converter para datetime apenas temporariamente para criar features
    dados['semana'] = pd.to_datetime(dados['semana'])
    
    print('   ✅ Target e merges concluídos.')
    
    # ============================================================================
    # ETAPA 3: ORDENAÇÃO E PREPARAÇÃO
    # ============================================================================
    print('   🔄 Ordenando dados para consistência temporal...')
    dados.sort_values(['pdv_id', 'produto_id', 'semana'], inplace=True)
    dados.reset_index(drop=True, inplace=True)
    
    gb_cols = ['pdv_id', 'produto_id']
    
    # ===========================================================================
    # ETAPA 4: FEATURES DE PREÇO COM DOWNCASTING IMEDIATO (VERSÃO CORRIGIDA)
    # ===========================================================================
    print('   💰 Criando features de preço (COM CORREÇÃO DE LEAKAGE)...')

    # 1. Calcule o preço unitário apenas onde a venda ocorreu, o resto fica NaN.
    # Isso quebra a ligação direta com o target 'vendeu'.
    preco_unitario_temp = np.where(
        dados['quantidade'] > 0,
        dados['faturamento'] / dados['quantidade'],
        np.nan  # Usar NaN em vez de 0 é a chave da correção!
    )
    dados['preco_unitario_atual'] = preco_unitario_temp.astype('float32')

    # 2. Propague o último preço conhecido para frente (forward fill) para cada item/loja.
    # Isso simula a realidade: o preço permanece o mesmo até ser alterado.
    # Usamos bfill() em seguida para cobrir casos onde as primeiras semanas são NaN.
    print('      propagando preços para semanas sem vendas...')
    dados['preco_unitario_atual'] = dados.groupby(gb_cols)['preco_unitario_atual'].transform(lambda x: x.ffill().bfill())

    # 3. Se ainda houver NaNs (itens que nunca venderam), preencha com 0.
    dados['preco_unitario_atual'].fillna(0, inplace=True)

    # Lags de preço (agora calculados sobre o preço corrigido e sem leakage)
    gb_preco = dados.groupby(gb_cols)['preco_unitario_atual']
    dados['preco_lag_1'] = gb_preco.shift(1).astype('float32')
    dados['preco_lag_2'] = gb_preco.shift(2).astype('float32')
    dados['variacao_preco_sku_semanal'] = (dados['preco_lag_1'] - dados['preco_lag_2']).fillna(0).astype('float32')

    print('   ✅ Features de preço criadas e corrigidas contra leakage.')
    
    # ============================================================================
    # ETAPA 5: FEATURES DE LAG E ROLLING (CRÍTICA) - CORREÇÃO TARGET LEAKAGE
    # ============================================================================
    print('   📊 Criando features de lag e rolling (ETAPA CRÍTICA - SEM TARGET LEAKAGE)...')

    gb_quantidade = dados.groupby(gb_cols)['quantidade']

    # Lags de quantidade como float32 (não float64)
    for lag in [1, 2, 3, 4]:
        dados[f'quantidade_lag_{lag}'] = gb_quantidade.shift(lag).astype('float32')

    # CORREÇÃO CRÍTICA: Calcular rolling features SOBRE OS DADOS DEFASADOS
    # Usar lag_1 como base para evitar que a semana atual influencie nas features
    gb_quantidade_lag1 = dados.groupby(gb_cols)['quantidade_lag_1']

    # Rolling features baseadas em dados do passado (sem a semana atual)
    rolling_window_corrigido = gb_quantidade_lag1.rolling(window=4, min_periods=1)
    dados['quantidade_media_4w'] = rolling_window_corrigido.mean().reset_index(level=gb_cols, drop=True).astype('float32')
    dados['quantidade_std_4w'] = rolling_window_corrigido.std().reset_index(level=gb_cols, drop=True).fillna(0).astype('float32')
    dados['quantidade_max_4w'] = rolling_window_corrigido.max().reset_index(level=gb_cols, drop=True).astype('float32')

    print('   ✅ Features de lag e rolling criadas SEM TARGET LEAKAGE.')
    
    # ============================================================================
    # ETAPA 6: FEATURES DA FASE 2 COM DOWNCASTING INCREMENTAL - CORREÇÃO EWMA
    # ============================================================================
    print('   🔄 Adicionando features EWMA (Fase 2 otimizada - SEM TARGET LEAKAGE)...')
    
    # CORREÇÃO CRÍTICA: EWMA também baseado em dados defasados
    dados['quantidade_ewma_4w'] = gb_quantidade_lag1.ewm(span=4, adjust=False).mean().reset_index(level=gb_cols, drop=True).astype('float32')
    dados['quantidade_ewma_8w'] = gb_quantidade_lag1.ewm(span=8, adjust=False).mean().reset_index(level=gb_cols, drop=True).astype('float32')
    dados['preco_ewma_4w'] = gb_preco.ewm(span=4, adjust=False).mean().reset_index(level=gb_cols, drop=True).astype('float32')
    
    print('   📅 Adicionando features de calendário avançadas (Fase 2 otimizada)...')
    dados['semana_do_ano'] = dados['semana'].dt.isocalendar().week.astype('int8')
    dados['eh_primeira_semana_mes'] = (dados['semana'].dt.day <= 7).astype('int8')
    dados['eh_dezembro'] = (dados['semana'].dt.month == 12).astype('int8')
    dados['eh_janeiro'] = (dados['semana'].dt.month == 1).astype('int8')
    dados['eh_pos_festas'] = ((dados['semana'].dt.month == 1) & (dados['semana'].dt.day <= 15)).astype('int8')
    
    # Features cíclicas como float32
    dados['semana_ano_sin'] = np.sin(2 * np.pi * dados['semana_do_ano'] / 52).astype('float32')
    dados['semana_ano_cos'] = np.cos(2 * np.pi * dados['semana_do_ano'] / 52).astype('float32')
    
    print('   💸 Adicionando features de preço relativo (Fase 2 otimizada)...')
    preco_medio_categoria = dados.groupby(['semana', 'categoria'])['preco_lag_1'].transform('mean')
    dados['preco_relativo_categoria'] = (dados['preco_lag_1'] / preco_medio_categoria).fillna(1.0).astype('float32')
    
    preco_medio_pdv = dados.groupby(['semana', 'pdv_id'])['preco_lag_1'].transform('mean')
    dados['preco_relativo_pdv'] = (dados['preco_lag_1'] / preco_medio_pdv).fillna(1.0).astype('float32')
    
    # CORREÇÃO CRÍTICA: preco_volatilidade baseada em features sem leakage
    dados['preco_volatilidade'] = (dados['quantidade_std_4w'] / (dados['quantidade_media_4w'] + 1e-6)).astype('float32')
    
    print('   ✅ Features da Fase 2 criadas e corrigidas contra target leakage.')
    
    # ============================================================================
    # ETAPA 7: FEATURES DE TENDÊNCIA E HIERARQUIA OTIMIZADAS - CORREÇÃO
    # ============================================================================
    print('   📈 Criando features de tendência e hierarquia (otimizadas - SEM TARGET LEAKAGE)...')
    
    dados['media_vendas_categoria_pdv_lag_1'] = dados.groupby(['semana', 'categoria', 'pdv_id'])['quantidade_lag_1'].transform('mean').astype('float32')
    vendas_categoria_pdv_lag = dados.groupby(['semana', 'categoria', 'pdv_id'])['quantidade_lag_1'].transform('sum')
    dados['share_vendas_sku_categoria_lag_1'] = (dados['quantidade_lag_1'] / vendas_categoria_pdv_lag).fillna(0).astype('float32')
    
    # CORREÇÃO CRÍTICA: momentum baseado em features sem leakage
    dados['momentum_ratio'] = (dados['quantidade_lag_1'] / dados['quantidade_media_4w']).fillna(0).astype('float32')
    dados['momentum_ratio_ewma'] = (dados['quantidade_lag_1'] / dados['quantidade_ewma_4w']).fillna(0).astype('float32')
    dados['aceleracao'] = (dados['quantidade_lag_1'] - dados['quantidade_lag_2']).fillna(0).astype('float32')
    
    print('   ✅ Features de tendência criadas SEM TARGET LEAKAGE.')
    
    # ============================================================================
    # ETAPA 8: FEATURES DE CALENDÁRIO BÁSICAS OTIMIZADAS
    # ============================================================================
    print('   📅 Criando features de calendário básicas (otimizadas)...')
    dados['dia_do_mes'] = dados['semana'].dt.day.astype('int8')
    dados['semana_do_mes'] = ((dados['semana'].dt.day - 1) // 7 + 1).astype('int8')
    dados['eh_inicio_mes'] = (dados['semana'].dt.day <= 7).astype('int8')
    dados['eh_fim_mes'] = (dados['semana'].dt.day >= 22).astype('int8')
    dados['mes'] = dados['semana'].dt.month.astype('int8')
    
    dados['mes_sin'] = np.sin(2 * np.pi * dados['mes'] / 12).astype('float32')
    dados['mes_cos'] = np.cos(2 * np.pi * dados['mes'] / 12).astype('float32')
    
    # ============================================================================
    # ETAPA 9: FEATURES PARA REGRESSÃO OTIMIZADAS
    # ============================================================================
    print('   🎯 Criando features específicas para REGRESSÃO (otimizadas)...')
    dados['preco_medio_semanal_sku_atual'] = dados.groupby(['semana', 'produto_id'])['preco_unitario_atual'].transform('mean').astype('float32')
    dados['media_vendas_categoria_pdv_atual'] = dados.groupby(['semana', 'categoria', 'pdv_id'])['quantidade'].transform('mean').astype('float32')
    vendas_categoria_pdv_atual = dados.groupby(['semana', 'categoria', 'pdv_id'])['quantidade'].transform('sum')
    dados['share_vendas_sku_categoria_atual'] = (dados['quantidade'] / vendas_categoria_pdv_atual).fillna(0).astype('float32')
    
    # ============================================================================
    # ETAPA 10: FEATURES CATEGÓRICAS HASH OTIMIZADAS
    # ============================================================================
    print('   🏷️ Criando features categóricas (otimizadas)...')
    dados['pdv_hash'] = (dados['pdv_id'].astype(str).apply(hash).abs() % 100).astype('int8')
    dados['produto_hash'] = (dados['produto_id'].astype(str).apply(hash).abs() % 100).astype('int8')
    dados['categoria_hash'] = (dados['categoria'].astype(str).apply(hash).abs() % 50).astype('int8')
    dados['zipcode_hash'] = (dados['zipcode'].astype(str).apply(hash).abs() % 1000).astype('int16')
    
    dados['pdv_produto_hash'] = (dados['pdv_hash'].astype('int16') * 100 + dados['produto_hash']).astype('int16')
    dados['categoria_zipcode_hash'] = (dados['categoria_hash'].astype('int32') * 1000 + dados['zipcode_hash']).astype('int32')
    
    # ============================================================================
    # ETAPA 11: LIMPEZA FINAL E REMOÇÃO DA COLUNA DATETIME PESADA
    # ============================================================================
    print('   🧹 Aplicando limpeza final e removendo coluna datetime pesada...')
    
    # CORREÇÃO: Preservar semana_original antes de remover a coluna datetime
    # Remover apenas a coluna datetime temporária, manter semana_original
    dados = dados.drop(columns=['semana'])
    
    # A coluna semana_original fica preservada para separação posterior
    
    # Limpeza de infinitos e NaNs
    dados.replace([np.inf, -np.inf], np.nan, inplace=True)
    dados.fillna(0, inplace=True)
    
    print(f'   ✅ {nome_conjunto} - Features criadas com sucesso: {dados.shape}')
    print('   🆕 Inclui todas as features da Fase 2: EWMA, calendário avançado, preço relativo')
    print('   🛡️ Memória otimizada a cada etapa - BLINDADO contra crash!')
    print('   🔒 Downcasting seguro aplicado - conversão automática de strings para numérico')
    print('   📅 Coluna semana_original preservada para separação treino/validação')
    print('   🔧 CORREÇÃO DE TARGET LEAKAGE APLICADA nas ETAPAS 5, 6 e 7')
    print('   ✅ Rolling, EWMA e features derivadas baseadas em quantidade_lag_1')
    
    return dados

print('✅ Função CORRIGIDA contra TARGET LEAKAGE definida!')

✅ Função CORRIGIDA contra TARGET LEAKAGE definida!


## 4. Aplicação da Engenharia de Features

In [5]:
# ==============================================================================
# VERSÃO CORRIGIDA - PROCESSAMENTO SEQUENCIAL PARA EVITAR CRASH DE MEMÓRIA
# ==============================================================================

# Função auxiliar para reduzir o uso de memória
def reduce_mem_usage(df, name):
    """
    Reduz o uso de memória de um dataframe, alterando os tipos de dados das colunas.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'🔄 Otimizando memória para {name}...')
    print(f'   📊 Memória inicial: {start_mem:.2f} MB')

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object and col_type.name != 'category' and 'datetime' not in col_type.name:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'   ✅ Memória final: {end_mem:.2f} MB ({100 * (start_mem - end_mem) / start_mem:.1f}% de redução)')
    return df

# Função para limpar tipos antes do salvamento - CORREÇÃO APLICADA AQUI
def fix_parquet_types(df):
    """Fix tipos problemáticos antes de salvar em parquet"""
    df_fixed = df.copy()
    
    # Converter todas as colunas object para string de forma robusta
    for col in df_fixed.columns:
        if df_fixed[col].dtype == 'object':
            # Primeiro converter todos valores para string
            df_fixed[col] = df_fixed[col].astype(str)
            # Limpar valores problemáticos
            df_fixed[col] = df_fixed[col].replace(['nan', 'None', '<NA>', 'NaN', 'null'], 'Unknown')
    
    return df_fixed

# --- NOVA LÓGICA DE EXECUÇÃO SEQUENCIAL ---

import gc

# Passo 1: Concatenar dados APENAS para criar features que dependem da série temporal completa (lags, rolling)
print('\n🔗 Etapa 1: Criando features temporais no dataset combinado...')
dados_completos = pd.concat([train_data, validation_data], ignore_index=True)

# Criar coluna semana_original para separar depois
dados_completos['semana_original'] = dados_completos['semana']

# CORREÇÃO CRÍTICA: Converter tipos dos dados auxiliares ANTES do merge
print('   🔧 Convertendo tipos dos dados auxiliares para compatibilidade...')

# Converter produto_id nos dados auxiliares para int64 (mesmo tipo do dados_completos)
if produtos['produto_id'].dtype == 'object':
    produtos['produto_id'] = pd.to_numeric(produtos['produto_id'], errors='coerce').astype('int64')
    print('   ✅ produtos[produto_id] convertido para int64')

if pdvs['pdv_id'].dtype == 'object':
    pdvs['pdv_id'] = pd.to_numeric(pdvs['pdv_id'], errors='coerce').astype('int64')
    print('   ✅ pdvs[pdv_id] convertido para int64')

# Limpar memória
del train_data, validation_data
gc.collect()

# Aplicar a função de engenharia de features
dados_completos_com_features = criar_features_avancadas_otimizada(dados_completos, produtos, pdvs, "DATASET_COMBINADO")

# Limpar memória
del dados_completos, produtos, pdvs
gc.collect()

print('   ✅ Features temporais criadas.')

# Passo 2: Separar novamente os conjuntos de treino e validação
print('\n📊 Etapa 2: Separando os datasets de volta para treino e validação...')
semanas_validacao = sorted(dados_completos_com_features['semana_original'].unique())[-5:]
train_features = dados_completos_com_features[~dados_completos_com_features['semana_original'].isin(semanas_validacao)].copy()
validation_features = dados_completos_com_features[dados_completos_com_features['semana_original'].isin(semanas_validacao)].copy()

# Remover coluna auxiliar
train_features = train_features.drop('semana_original', axis=1)
validation_features = validation_features.drop('semana_original', axis=1)

# Passo CRÍTICO: Deletar o dataframe gigante e forçar a limpeza de memória
print('   🗑️ Liberando memória do dataframe combinado...')
del dados_completos_com_features
gc.collect()
print('   ✅ Memória liberada.')

# Passo 3: Otimizar a memória de cada dataframe SEPARADAMENTE
train_features = reduce_mem_usage(train_features, "train_features")
validation_features = reduce_mem_usage(validation_features, "validation_features")

# Passo 4: Salvar os arquivos um de cada vez - COM CORREÇÃO DE TIPOS
print('\n💾 Etapa Final: Salvando arquivos otimizados sequencialmente...')

# CORREÇÃO: Aplicar fix de tipos antes de salvar
print('   🔧 Aplicando correção de tipos para compatibilidade com Parquet...')
train_features = fix_parquet_types(train_features)
validation_features = fix_parquet_types(validation_features)

# Salvar dados de treino
output_path_train = '../data/submissao3/train_features.parquet'
train_features.to_parquet(output_path_train, index=False)
print(f'   ✅ Dados de TREINO salvos em: {output_path_train}')
print(f'      Shape: {train_features.shape}, Colunas: {len(train_features.columns)}')

# Liberar memória do treino antes de salvar a validação
del train_features
gc.collect()

# Salvar dados de validação
output_path_validation = '../data/submissao3/validation_features.parquet'
validation_features.to_parquet(output_path_validation, index=False)
print(f'   ✅ Dados de VALIDAÇÃO salvos em: {output_path_validation}')
print(f'      Shape: {validation_features.shape}, Colunas: {len(validation_features.columns)}')

# Liberar memória da validação
del validation_features
gc.collect()

# Salvar metadados (código original mantido)
print('\n📋 Salvando metadados das features...')
import pickle

metadados_features = {
    'data_criacao': pd.Timestamp.now(),
    'estrategia': 'Engenharia de Features Avançada para Modelo de Dois Estágios',
    'total_features': len(pd.read_parquet(output_path_train).columns),
    'target_classificacao': 'vendeu (1 se quantidade > 0, 0 caso contrário)',
    'target_regressao': 'quantidade (apenas onde vendeu = 1)',
    'tipos_features': [
        'Target "vendeu" para classificação',
        'Features de preço (unitário, lags, médias)',
        'Features de calendário (semana, mês)',
        'Features de lag de vendas (1 a 4 semanas)',
        'Features de rolling window (médias, std dev)',
        'Features EWMA (médias ponderadas exponencialmente)',
        'Features de hierarquia (agregados por categoria/zipcode)',
        'Features de tendência como momentum e aceleração',
        'Features hash criam embeddings para variáveis categóricas'
    ],
    'observacoes': [
        'Processamento sequencial para evitar esgotamento de memória',
        'Otimização de tipos de dados aplicada',
        'Features de Fase 2 incluídas: EWMA, calendário avançado, preço relativo',
        'Compatibilidade de tipos garantida para merges',
        'Correção de tipos aplicada para salvamento em Parquet'
    ]
}

with open('../data/submissao3/advanced_features_metadata.pkl', 'wb') as f:
    pickle.dump(metadados_features, f)

print('   ✅ Metadados salvos em: data/submissao3/advanced_features_metadata.pkl')

print('\n🎉 ENGENHARIA DE FEATURES AVANÇADA CONCLUÍDA (COM OTIMIZAÇÃO DE MEMÓRIA)!')
print('=' * 80)
print('🎯 Features criadas para modelo de dois estágios:')
print('   📊 CLASSIFICAÇÃO: Use target "vendeu" para treinar se vai vender')
print('   📊 REGRESSÃO: Use target "quantidade" (apenas onde vendeu=1) para quanto vai vender')
print('\n💡 Próximos passos:')
print('   1. Treinar modelo de classificação (LGBMClassifier)')
print('   2. Treinar modelo de regressão (LGBMRegressor)')
print('   3. Combinar os dois no pipeline final')
print('\n🚀 Dados prontos para modelagem avançada!')


🔗 Etapa 1: Criando features temporais no dataset combinado...
   🔧 Convertendo tipos dos dados auxiliares para compatibilidade...
   ✅ produtos[produto_id] convertido para int64
   ✅ pdvs[pdv_id] convertido para int64
\n🔧 Criando features avançadas para DATASET_COMBINADO (VERSÃO CORRIGIDA - SEM TARGET LEAKAGE)...
   🎯 Otimização inicial de tipos de dados (verificando limites)...
      🔍 pdv_id: tipo=object
         Exemplos: ['1008240744247283174', '1008240744247283174', '10097752152132198']
   🔄 Coluna pdv_id: convertida de string para numérico
   ⚠️ Coluna pdv_id: valores muito grandes para int32, mantendo int64
      🔍 produto_id: tipo=object
         Exemplos: ['1938760505411922162', '4098058333001424920', '1938760505411922162']
   🔄 Coluna produto_id: convertida de string para numérico
   ⚠️ Coluna produto_id: valores muito grandes para int32, mantendo int64
      🔍 distributor_id: tipo=float32
         Range: 4.0 até 11.0
   ✅ Otimização inicial concluída.
   🎯 Criando target e 

In [6]:
# Analisar features criadas SEM TARGET LEAKAGE
print('📊 Analisando features criadas SEM TARGET LEAKAGE...')

# Carregar os dados salvos para análise (já que foram deletados para economizar memória)
print('📂 Carregando dados salvos para análise...')
train_features = pd.read_parquet('../data/submissao3/train_features.parquet')
validation_features = pd.read_parquet('../data/submissao3/validation_features.parquet')

# Listar todas as features
features_disponiveis = list(train_features.columns)
print(f'\n📋 Features disponíveis ({len(features_disponiveis)}):')

# Categorizar features CORRIGIDAS
features_categorias = {
    'Básicas': ['semana', 'pdv_id', 'produto_id', 'quantidade', 'faturamento', 'distributor_id'],
    'Target': ['vendeu'],
    'Auxiliares': ['categoria', 'zipcode', 'tipo_loja'],
    'Preço_SEM_Leakage': [col for col in features_disponiveis if 'preco' in col and 'lag' in col],
    'Preço_PARA_Regressão': [col for col in features_disponiveis if 'preco' in col and 'atual' in col],
    'Calendário': [col for col in features_disponiveis if any(x in col for x in ['dia', 'semana_do', 'mes', 'inicio', 'fim'])],
    'Lag': [col for col in features_disponiveis if 'lag_' in col and 'preco' not in col],
    'Rolling': [col for col in features_disponiveis if any(x in col for x in ['media_4w', 'std_4w', 'max_4w'])],
    'Hierarquia_SEM_Leakage': [col for col in features_disponiveis if any(x in col for x in ['media_vendas', 'share']) and 'lag' in col],
    'Hierarquia_PARA_Regressão': [col for col in features_disponiveis if any(x in col for x in ['media_vendas', 'share']) and 'atual' in col],
    'Tendência': [col for col in features_disponiveis if any(x in col for x in ['momentum', 'aceleracao'])],
    'Hash': [col for col in features_disponiveis if 'hash' in col]
}

for categoria, features in features_categorias.items():
    if features:
        print(f'\n🏷️ {categoria} ({len(features)}):')
        for feat in features:
            if feat in features_disponiveis:
                print(f'   • {feat}')

print('\n🚨 ANÁLISE DE TARGET LEAKAGE:')
features_sem_leakage = []
features_com_leakage = []

for feat in features_disponiveis:
    if feat in ['semana', 'pdv_id', 'produto_id', 'quantidade', 'faturamento', 'distributor_id', 'vendeu']:
        continue  # Básicas, não contar
    elif 'atual' in feat:
        features_com_leakage.append(feat)  # Para regressão apenas
    else:
        features_sem_leakage.append(feat)  # Seguras para classificação

print(f'✅ Features SEGURAS para classificação: {len(features_sem_leakage)}')
for feat in features_sem_leakage[:10]:  # Mostrar apenas primeiras 10
    print(f'   • {feat}')
if len(features_sem_leakage) > 10:
    print(f'   • ... e mais {len(features_sem_leakage)-10} features')

print(f'\n⚠️ Features COM LEAKAGE (apenas para regressão): {len(features_com_leakage)}')
for feat in features_com_leakage:
    print(f'   • {feat}')

print('\n📊 Estatísticas do conjunto de treino (SEM LEAKAGE):')
print(f'   • Shape: {train_features.shape}')
print(f'   • Target vendeu: {train_features["vendeu"].sum():,} positivos ({train_features["vendeu"].mean()*100:.1f}%)')
print(f'   • Quantidade média: {train_features["quantidade"].mean():.2f}')
print(f'   • Registros com lag_4: {(train_features["quantidade_lag_4"] > 0).sum():,}')

print('\n📊 Estatísticas do conjunto de validação (SEM LEAKAGE):')
print(f'   • Shape: {validation_features.shape}')
print(f'   • Target vendeu: {validation_features["vendeu"].sum():,} positivos ({validation_features["vendeu"].mean()*100:.1f}%)')
print(f'   • Quantidade média: {validation_features["quantidade"].mean():.2f}')
print(f'   • Registros com lag_4: {(validation_features["quantidade_lag_4"] > 0).sum():,}')

print('✅ Análise das features SEM TARGET LEAKAGE concluída')

📊 Analisando features criadas SEM TARGET LEAKAGE...
📂 Carregando dados salvos para análise...

📋 Features disponíveis (54):

🏷️ Básicas (6):
   • pdv_id
   • produto_id
   • quantidade
   • faturamento
   • distributor_id

🏷️ Target (1):
   • vendeu

🏷️ Auxiliares (3):
   • categoria
   • zipcode
   • tipo_loja

🏷️ Preço_SEM_Leakage (2):
   • preco_lag_1
   • preco_lag_2

🏷️ Preço_PARA_Regressão (2):
   • preco_unitario_atual
   • preco_medio_semanal_sku_atual

🏷️ Calendário (12):
   • quantidade_media_4w
   • semana_do_ano
   • eh_primeira_semana_mes
   • media_vendas_categoria_pdv_lag_1
   • dia_do_mes
   • semana_do_mes
   • eh_inicio_mes
   • eh_fim_mes
   • mes
   • mes_sin
   • mes_cos
   • media_vendas_categoria_pdv_atual

🏷️ Lag (6):
   • quantidade_lag_1
   • quantidade_lag_2
   • quantidade_lag_3
   • quantidade_lag_4
   • media_vendas_categoria_pdv_lag_1
   • share_vendas_sku_categoria_lag_1

🏷️ Rolling (3):
   • quantidade_media_4w
   • quantidade_std_4w
   • quantidade_max

In [7]:
# ============================================================================
# REGENERAR FEATURES CORRIGIDAS - SEM TARGET LEAKAGE
# ============================================================================

print('🔧 Regenerando features com CORREÇÃO de target leakage...')

# Limpar dados anteriores
import gc
if 'train_features' in locals():
    del train_features
if 'validation_features' in locals():
    del validation_features
gc.collect()

# Carregar dados originais novamente
print('📂 Recarregando dados originais...')
train_data = pd.read_parquet('../data/submissao3/train_data.parquet')
validation_data = pd.read_parquet('../data/submissao3/validation_data.parquet')

# Carregar dados auxiliares
produtos = pd.read_parquet(
    '../data/part-00000-tid-7173294866425216458-eae53fbf-d19e-4130-ba74-78f96b9675f1-4-1-c000.snappy.parquet'
)
produtos = produtos.rename(columns={'produto': 'produto_id'})

pdvs = pd.read_parquet(
    '../data/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet'
)
pdvs = pdvs.rename(columns={'pdv': 'pdv_id', 'categoria_pdv': 'tipo_loja'})

# Converter tipos dos dados auxiliares
if produtos['produto_id'].dtype == 'object':
    produtos['produto_id'] = pd.to_numeric(produtos['produto_id'], errors='coerce').astype('int64')
if pdvs['pdv_id'].dtype == 'object':
    pdvs['pdv_id'] = pd.to_numeric(pdvs['pdv_id'], errors='coerce').astype('int64')

# Combinar dados para criar features temporais consistentes
print('🔗 Combinando dados para criar features temporais...')
dados_completos = pd.concat([train_data, validation_data], ignore_index=True)
dados_completos['semana_original'] = dados_completos['semana']

del train_data, validation_data
gc.collect()

# Aplicar a função CORRIGIDA
print('⚡ Aplicando engenharia de features CORRIGIDA...')
dados_completos_corrigidos = criar_features_avancadas_otimizada(
    dados_completos, produtos, pdvs, "DATASET_CORRIGIDO"
)

del dados_completos, produtos, pdvs
gc.collect()

# Separar novamente
print('📊 Separando datasets corrigidos...')
semanas_validacao = sorted(dados_completos_corrigidos['semana_original'].unique())[-5:]
train_features_corrigido = dados_completos_corrigidos[
    ~dados_completos_corrigidos['semana_original'].isin(semanas_validacao)
].copy()
validation_features_corrigido = dados_completos_corrigidos[
    dados_completos_corrigidos['semana_original'].isin(semanas_validacao)
].copy()

# Remover coluna auxiliar
train_features_corrigido = train_features_corrigido.drop('semana_original', axis=1)
validation_features_corrigido = validation_features_corrigido.drop('semana_original', axis=1)

del dados_completos_corrigidos
gc.collect()

# Otimizar memória
train_features_corrigido = reduce_mem_usage(train_features_corrigido, "train_features_corrigido")
validation_features_corrigido = reduce_mem_usage(validation_features_corrigido, "validation_features_corrigido")

# Salvar dados corrigidos
print('💾 Salvando dados CORRIGIDOS...')

def fix_parquet_types(df):
    """Fix tipos problemáticos antes de salvar em parquet"""
    df_fixed = df.copy()
    for col in df_fixed.columns:
        if df_fixed[col].dtype == 'object':
            df_fixed[col] = df_fixed[col].astype(str)
            df_fixed[col] = df_fixed[col].replace(['nan', 'None', '<NA>', 'NaN', 'null'], 'Unknown')
    return df_fixed

train_features_corrigido = fix_parquet_types(train_features_corrigido)
validation_features_corrigido = fix_parquet_types(validation_features_corrigido)

# Salvar arquivos corrigidos
train_features_corrigido.to_parquet('../data/submissao3/train_features_CORRIGIDO.parquet', index=False)
validation_features_corrigido.to_parquet('../data/submissao3/validation_features_CORRIGIDO.parquet', index=False)

print('✅ Arquivos CORRIGIDOS salvos:')
print('   • data/submissao3/train_features_CORRIGIDO.parquet')
print('   • data/submissao3/validation_features_CORRIGIDO.parquet')

# Análise rápida das diferenças
print('\\n📊 ANÁLISE COMPARATIVA - ANTES vs DEPOIS DA CORREÇÃO:')
print(f'📐 Shape treino: {train_features_corrigido.shape}')
print(f'📐 Shape validação: {validation_features_corrigido.shape}')

# Testar algumas features críticas
exemplos_features = ['quantidade_media_4w', 'quantidade_std_4w', 'quantidade_ewma_4w', 'momentum_ratio']
print('\\n🔍 Análise de features críticas (primeiros 10 registros):')

for feat in exemplos_features:
    if feat in train_features_corrigido.columns:
        valores = train_features_corrigido[feat].head(10)
        print(f'   {feat}: {valores.tolist()[:5]} ...')

print('\\n🎯 Estatísticas do target:')
print(f'   Treino vendeu=1: {train_features_corrigido["vendeu"].sum():,} ({train_features_corrigido["vendeu"].mean()*100:.1f}%)')
print(f'   Validação vendeu=1: {validation_features_corrigido["vendeu"].sum():,} ({validation_features_corrigido["vendeu"].mean()*100:.1f}%)')

# Salvar metadados da correção
import pickle
metadados_correcao = {
    'data_correcao': pd.Timestamp.now(),
    'versao': 'CORRIGIDA - SEM TARGET LEAKAGE',
    'principais_correcoes': [
        'Rolling features baseadas em quantidade_lag_1 (não quantidade atual)',
        'EWMA features baseadas em quantidade_lag_1 (não quantidade atual)', 
        'Features derivadas (momentum, volatilidade) usam features corrigidas',
        'Separação clara entre features para classificação vs regressão'
    ],
    'features_seguras_classificacao': [f for f in train_features_corrigido.columns 
                                       if not any(x in f for x in ['atual', 'quantidade']) or 'lag' in f],
    'total_features': len(train_features_corrigido.columns)
}

with open('../data/submissao3/correcao_target_leakage_metadata.pkl', 'wb') as f:
    pickle.dump(metadados_correcao, f)

print('📋 Metadados da correção salvos!')
print('\\n🎉 CORREÇÃO DE TARGET LEAKAGE CONCLUÍDA!')
print('=' * 70)
print('✅ Próximo passo: Re-treinar o modelo de classificação com dados corrigidos')
print('📈 Espere métricas mais realistas (AUC ~0.85-0.95, F1 ~0.6-0.8)')

# Limpar variáveis para economizar memória
del train_features_corrigido, validation_features_corrigido
gc.collect()

print('🧹 Memória limpa. Pronto para próximos passos!')

🔧 Regenerando features com CORREÇÃO de target leakage...
📂 Recarregando dados originais...
🔗 Combinando dados para criar features temporais...
⚡ Aplicando engenharia de features CORRIGIDA...
\n🔧 Criando features avançadas para DATASET_CORRIGIDO (VERSÃO CORRIGIDA - SEM TARGET LEAKAGE)...
   🎯 Otimização inicial de tipos de dados (verificando limites)...
      🔍 pdv_id: tipo=object
         Exemplos: ['1008240744247283174', '1008240744247283174', '10097752152132198']
   🔄 Coluna pdv_id: convertida de string para numérico
   ⚠️ Coluna pdv_id: valores muito grandes para int32, mantendo int64
      🔍 produto_id: tipo=object
         Exemplos: ['1938760505411922162', '4098058333001424920', '1938760505411922162']
   🔄 Coluna produto_id: convertida de string para numérico
   ⚠️ Coluna produto_id: valores muito grandes para int32, mantendo int64
      🔍 distributor_id: tipo=float32
         Range: 4.0 até 11.0
   ✅ Otimização inicial concluída.
   🎯 Criando target e fazendo merge com dados auxi